# Final Project: Huggingface Transformers를 이용한 한-영 번역

### 개요
- 한-영 번역 프로그램을 Huggingface의 Transformers와 Dataset을 이용해서 필요한 크기로 데이터를 줄이고 이를 모델에서 파인튜닝하여 번역 성능을 살펴보고 실제 예문의 번역을 출력
- [Huggingface NLP course의 7장 Translation](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt)을 근간으로 해서 (거의 그대로 활용할 수 있음) 구현할 수 있음
- Dataset을 자료를 받아서 필요한 크기로 나누고, 학습에 필요한 형태로 Dataset을 재구조화하고 tokenize하는 모듈을 구현
- 공개된 자료를 바탕으로 구현하기 때문에 성능보다는 전체 번역모듈을 Huggingface로 구현해보는 것을 주목표로 하기 때문에 완결성이 있어야 하며, 실제로 작동해야 함.
- 파일 이름 NLPProject_이름/그룹.ipynb
- Due 3월 9일 11시 59분

## 필요한 모듈 설치
- 프로그램 실행에 필요한 모듈, Huggingface, Dataset 등을 각자 알아서 설치

In [3]:
!pip install transformers

In [4]:
!pip install datasets

  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohappyeyeballs-2.5.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached datasets-3.3.2-py3-none-any.whl (485 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.5 MB 12.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.4/25.5 MB 10.1 MB/s eta 0:00:03
   -------- ------------------------------- 5.5/25.5 MB 9.1 MB/s eta 0:00:03
   ------

In [5]:
# T5 모델이 sentence piece tokenizer를 사용하기 때문에 데이터를 이 형태로 토크나이즈 하기 위해
!pip install sentencepiece

In [6]:
!pip install accelerate

  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.4.0-py3-none-any.whl (342 kB)


In [7]:
# 모델 평가를 위해
!pip install evaluate

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)


In [8]:
#bleu 스코어 계산을 위해
!pip install sacrebleu

  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached portalocker-3.1.1-py3-none-any.whl.metadata (8.6 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached sacrebleu-2.5.1-py3-none-any.whl (104 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------------ --------------- 2.4/3.8 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 11.4 MB/s eta 0:00:00
Using cached portalocker-3.1.1-py3-none-any.whl (19 kB)


In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
device


2.7.0.dev20250306+cu128


device(type='cuda')

## Dataset
- Huggingface Hub에 있는 Dataset 중 `bongsoo/news_talk_en_ko` 는 한국어-영어뉴스 기사를 병렬로 작성한 130만 개의 데이터 셋이다.
- 이 데이터셋을 읽어서 colab에서 돌릴 수 있게, training, validation, test 데이터로 각각 120,000, 9,000, 1,000으로 줄여서 학습에 필요한 구조로 만듬
- 데이터를 자를때 순차적으로 자르지 말고 전체 데이터를 셔플한 후 필요한 크기로 자를 것
- 데이터셋을 pandas 형식으로 받은 후 할 수도 있고 여러 가능한 방법이 있음

In [1]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset

en_ko = load_dataset("bongsoo/news_talk_en_ko")

c:\Users\ehrbs\miniconda3\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [3]:
# 허깅페이스 데이터셋을 판다스 포맷으로 세팅
en_ko.set_format(type="pandas")

In [4]:
# 'train'키의 모든 행을 DataFrame df에 할당
df = en_ko["train"][:]

df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [5]:
col = list(df.columns)
col

["Skinner's reward is mostly eye-watering.",
 '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.']

In [6]:
example_0_df = pd.DataFrame({col: [value] for col, value in zip(('en', 'ko'), col)})

In [7]:
df.columns = ('en', 'ko')

In [8]:
en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [9]:
en_ko = Dataset.from_pandas(en_ko_df)

In [10]:
en_ko

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [11]:
data = en_ko.shuffle(seed=42)  # 셔플 적용

train_size = 120000
valid_size = 9000
test_size = 1000

In [12]:
train_data = data.select(range(train_size))
valid_data = data.select(range(train_size, train_size + valid_size))
test_data = data.select(range(train_size + valid_size, train_size + valid_size + test_size))

# 데이터셋 형태로 변환
split_datasets = {
    "train": train_data,
    "validation": valid_data,
    "test": test_data
}


In [13]:

split_datasets


{'train': Dataset({
     features: ['en', 'ko'],
     num_rows: 120000
 }),
 'validation': Dataset({
     features: ['en', 'ko'],
     num_rows: 9000
 }),
 'test': Dataset({
     features: ['en', 'ko'],
     num_rows: 1000
 })}

In [14]:
split_datasets["train"][:2]

{'en': ['Suncheon National University said on the 13th, "President Park Jin-sung tendered his resignation to take responsibility for being included in a \'capacity-enhancing university\' that requires reducing the number of new students in the ministry\'s college evaluation.',
  'Those are scabs, don’t pick them with your hands, but wait until they come off naturally.'],
 'ko': ['순천대는 13일 “박진성 총장이 교육부 대학평가에서 신입생 정원을 줄여야 하는 ‘역량강화대학’에 포함된 것에 대해 책임을 지고 사표를 제출했다”고 밝혔다.',
  '딱지인데 손으로 떼지 마시고 떨어질 때까지 기다리세요.']}

## Huggingface
- 학습에 필요한 Huggingface 모델 사용
- AutoTokenizer, AutoModelForSeq2SeqLM 등을 사용
- 학습에 사용할 모델은 [T5](https://github.com/AIRC-KETI/ke-t5)("KETI-AIR/ke-t5-base")를 사용할 것
- T5모델은 트랜스포머의 인코더, 디코더 구조를 모두 사용하는 모델로 번역기를 만들 때 사용할 수 있는 모델이다.
- 아래처럼 모델 체크 포인트와 T5 모델에 입력될 최대 토큰 길이를 적절히 설정한다.

In [15]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [16]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_length = 64

## Tokenizer
- T5는 sentencepiece tokenizer를 사용하기 때문에 한-영 병렬 데이터의 자료를 학습시키기 위해서는 이 데이터를 tokenizer를 써서 프로세싱을 해야 한다. 이를 위한 모듈을 만들고 한국어, 영어데이터를 tokenize하여 모델에 입력할 수 있는 형태로(tokenized-dataset) 바꾼다
- 이를 위해서 Dataset의 map()을 활용하도록 한다.

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [18]:
en_sentence = split_datasets["train"][0]["en"]
ko_sentence = split_datasets["train"][0]["ko"]

inputs = tokenizer(ko_sentence, text_target=en_sentence)
inputs

{'input_ids': [13362, 4249, 398, 30, 36, 597, 214, 192, 24570, 7486, 1285, 2606, 37, 17117, 44419, 43985, 153, 42, 15303, 13593, 2059, 24, 9, 7599, 1715, 151, 2763, 14856, 32510, 12156, 39, 25, 108, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [7417, 8265, 822, 1655, 1444, 87, 46, 5, 398, 557, 4, 23, 52962, 2512, 1055, 476, 28, 8, 5291, 29554, 102, 111, 29746, 10, 624, 8914, 40, 519, 3779, 20, 16, 33, 14900, 4334, 1450, 28, 1264, 1171, 22282, 8862, 17, 38, 9361, 17147, 5, 1090, 14, 241, 1242, 20, 5, 17288, 17, 8, 4292, 26700, 3, 1]}

In [19]:
wrong_targets = tokenizer(ko_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁순천', '대는', '▁13', '일', '▁“', '박', '진', '성', '▁총장이', '▁교육부', '▁대학', '평가', '에서', '▁신입생', '▁정원을', '▁줄여야', '▁하는', '▁‘', '역량', '강화', '대학', '’', '에', '▁포함된', '▁것에', '▁대해', '▁책임을', '▁지고', '▁사표를', '▁제출했다', '”', '고', '▁밝혔다', '.', '</s>']
['▁Sun', 'che', 'on', '▁National', '▁University', '▁said', '▁on', '▁the', '▁13', 'th', ',', '▁"', 'President', '▁Park', '▁J', 'in', '-', 's', 'ung', '▁tender', 'ed', '▁his', '▁resignation', '▁to', '▁take', '▁responsibility', '▁for', '▁being', '▁included', '▁in', '▁a', "▁'", 'cap', 'ac', 'ity', '-', 'en', 'h', 'ancing', '▁university', "'", '▁that', '▁requires', '▁reducing', '▁the', '▁number', '▁of', '▁new', '▁students', '▁in', '▁the', '▁ministry', "'", 's', '▁college', '▁evaluation', '.', '</s>']


In [20]:
def preprocess_function(examples):
    inputs = examples["ko"]
    targets = examples["en"]

    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


In [21]:
from datasets import DatasetDict

split_datasets = DatasetDict({
    "train": split_datasets["train"],
    "validation": split_datasets["validation"],
    "test": split_datasets["test"],
})


In [22]:
print(split_datasets["train"])


Dataset({
    features: ['en', 'ko'],
    num_rows: 120000
})


In [23]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=['ko', 'en'],  # 정확한 컬럼명으로 수정
)


Map: 100%|██████████| 1000/1000 [00:00<00:00, 23256.08 examples/s]


In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [25]:
tokenized_datasets["train"][1]

{'input_ids': [4475,
  76,
  1166,
  8383,
  48562,
  14173,
  13016,
  2956,
  13073,
  6058,
  3,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [6563,
  69,
  9778,
  5697,
  8,
  4,
  697,
  24,
  71,
  5032,
  342,
  54,
  273,
  4977,
  4,
  134,
  6550,
  1429,
  128,
  941,
  590,
  30260,
  3,
  1]}

In [26]:
print( '원 데이터    :', split_datasets['train'][10]['ko'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', split_datasets['train'][10]['en'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['labels'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )

원 데이터    : 해경은 불법포획 흔적이 발견되지 않자 사체를 읍사무소에 인계했다.
처리 후 데이터: [32944, 1635, 771, 17180, 25682, 32220, 18619, 37014, 21, 18192, 9543, 9, 43829, 107, 3, 1]
토큰화       : ['▁해경은', '▁불법', '포', '획', '▁흔적이', '▁발견되지', '▁않자', '▁사체', '를', '▁읍', '사무소', '에', '▁인계', '했다', '.', '</s>']


원 데이터    : When the coast guard found no trace of illegal capture, they handed over the dead body to the town office.
처리 후 데이터: [1319, 5, 12343, 10211, 1019, 353, 28241, 14, 8322, 15180, 4, 128, 19885, 246, 5, 5688, 3138, 10, 5, 3415, 1803, 3, 1]
토큰화       : ['▁When', '▁the', '▁coast', '▁guard', '▁found', '▁no', '▁trace', '▁of', '▁illegal', '▁capture', ',', '▁they', '▁handed', '▁over', '▁the', '▁dead', '▁body', '▁to', '▁the', '▁town', '▁office', '.', '</s>']


## Model
- 학습에 필요한 모델 설정

In [27]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

## Collator
- 학습할 자료를 정렬하고 모델에 배치 단위로 넘겨주기 위해 준비

In [28]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [30]:
batch["labels"]

tensor([[   55,   634, 31343, 10833, 20450,  6399,  3501, 14147, 30767,  5896,
          2017,    16, 16514, 33606,    46,  7860, 12391, 46756, 12979,    65,
             5,   384, 12109,  5097,    28, 34817,  1601,  4555,    46,     5,
            77,   557,     4,    54,   322,   235,   238,   738,  1707,    73,
          2753, 19701,    20,     5, 13063,  1415,    14, 18015, 38744, 18871,
            28,  3476,    13,   515,    73, 30292,  1368,    14, 12391, 46756,
         12979,     3,     1],
        [  223,    62,     5,   275,  5349,  3186,  5517,  2228,    14,     5,
           256,  2017,    20,  3181,   236,     5,  5349,  3186, 16982,  1934,
            16, 19699,   482,  6413,    38,     5, 18068,   615,  1990,   149,
         20729,   655,   366,   256,     3,     1,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100

In [31]:
batch["decoder_input_ids"]

tensor([[    0,    55,   634, 31343, 10833, 20450,  6399,  3501, 14147, 30767,
          5896,  2017,    16, 16514, 33606,    46,  7860, 12391, 46756, 12979,
            65,     5,   384, 12109,  5097,    28, 34817,  1601,  4555,    46,
             5,    77,   557,     4,    54,   322,   235,   238,   738,  1707,
            73,  2753, 19701,    20,     5, 13063,  1415,    14, 18015, 38744,
         18871,    28,  3476,    13,   515,    73, 30292,  1368,    14, 12391,
         46756, 12979,     3],
        [    0,   223,    62,     5,   275,  5349,  3186,  5517,  2228,    14,
             5,   256,  2017,    20,  3181,   236,     5,  5349,  3186, 16982,
          1934,    16, 19699,   482,  6413,    38,     5, 18068,   615,  1990,
           149, 20729,   655,   366,   256,     3,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0

In [32]:
for i in range(1, 3):
    print(tokenized_datasets["validation"][i]["labels"])

[55, 634, 31343, 10833, 20450, 6399, 3501, 14147, 30767, 5896, 2017, 16, 16514, 33606, 46, 7860, 12391, 46756, 12979, 65, 5, 384, 12109, 5097, 28, 34817, 1601, 4555, 46, 5, 77, 557, 4, 54, 322, 235, 238, 738, 1707, 73, 2753, 19701, 20, 5, 13063, 1415, 14, 18015, 38744, 18871, 28, 3476, 13, 515, 73, 30292, 1368, 14, 12391, 46756, 12979, 3, 1]
[223, 62, 5, 275, 5349, 3186, 5517, 2228, 14, 5, 256, 2017, 20, 3181, 236, 5, 5349, 3186, 16982, 1934, 16, 19699, 482, 6413, 38, 5, 18068, 615, 1990, 149, 20729, 655, 366, 256, 3, 1]


## Metric
- 학습한 모델을 측정할 매트릭을 준비
- 번역 모델에서는 주로 BLEU 점수를 사용
- BLEU 점수는 번역기가 생성한 문장이 레퍼런스(정답이라는 표현을 사용하지 않는 이유는 제대로 된 번역 문장이 오직 하나가 아니기 때문)문장과 얼마나 비슷한지 측정하는 점수

- sacrebleu 라이브러리는 BLEU 구현체에서 사실상 표준 라이브러리이며 각 모델이 다른 토크나이저를 쓰는 경우 이를 BPE로 통일 시켜 BLEU 점수를 계산



In [33]:
import evaluate
import numpy as np
metric = evaluate.load("sacrebleu")

In [34]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

## 모델 학습(Train)
- 학습을 간단히 하기위해 허깅페이스에서 제공하는 Seq2SeqTrainer클래스와 학습 세부 조건은 Seq2SeqTrainingArguments를 활용할 수 있으나, 본 과제에서는 이를 쓰지 말고 Training를 직접 구현하도록 한다. Dataloader, Scheduler, ACCELERATOR, Optimizer 등을 설정하고 실제로 training loop를 돌려서 학습하고, evaluation 데이터로 성능을 검증
- colab에서 돌리기 위해서는 성능이 저하되겠지만, batch size 등을 적당하게 설정해야 함.

In [35]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import os

In [45]:
import torch

# GPU 캐시 초기화
torch.cuda.empty_cache()

In [47]:
# checkpoint 경로 지정
checkpoint_path = "./results"

In [48]:
# 체크포인트가 존재하면 모델 불러오기, 없으면 새로 초기화 (여기서는 체크포인트가 있다고 가정)
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}")
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
else:
    # 모델 초기화 코드 (예: model = MyModelClass.from_pretrained("pretrained-model-name"))
    raise ValueError("Checkpoint not found. Please provide a valid checkpoint path.")

Loading checkpoint from ./results


In [49]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [52]:
# 하이퍼파라미터 설정 (Colab 메모리 제약에 맞게 배치 사이즈 등 조정)
num_epochs = 5
gradient_accumulation_steps = 2
train_batch_size = 16    # Colab 환경에 맞게 조정 (원래 예제는 64)
eval_batch_size = 32     # 원래 예제는 128
learning_rate = 5e-4
weight_decay = 0.01
save_steps = 500

In [53]:
# 얼리 스토핑 관련 설정
patience = 3  # 개선이 없을 때 최대 에폭 수
best_eval_loss = float("inf")
patience_counter = 0

In [54]:
# DataLoader 생성 (전처리된 데이터셋 사용)
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=eval_batch_size,
    shuffle=False,
    collate_fn=data_collator
)

In [55]:
# Optimizer 설정
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

c:\Users\ehrbs\miniconda3\envs\new_env\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [56]:
# 전체 학습 스텝 계산 및 Scheduler 설정
total_training_steps = (len(train_dataloader) // gradient_accumulation_steps) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_training_steps)

In [57]:
for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", leave=True)
    
    optimizer.zero_grad()
    for step, batch in enumerate(progress_bar):
        # batch를 device로 이동
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # 모델 forward pass (loss 포함)
        outputs = model(**batch)
        loss = outputs.loss
        
        # gradient accumulation 적용
        loss = loss / gradient_accumulation_steps
        loss.backward()
        
        epoch_loss += loss.item()
        
        # 누적된 스텝마다 optimizer update
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
        
        progress_bar.set_postfix(loss=loss.item() * gradient_accumulation_steps)
    
    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Training Loss: {avg_loss:.4f}")
    
    # Evaluation loop
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating", leave=True):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            eval_loss += outputs.loss.item()
    avg_eval_loss = eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch+1} Evaluation Loss: {avg_eval_loss:.4f}")
    
    # 평가 손실이 개선되었을 때 덮어쓰기
    if avg_eval_loss < best_eval_loss:
        best_eval_loss = avg_eval_loss
        patience_counter = 0
        model.save_pretrained("./best_model")
        print(f"Epoch {epoch+1}: Evaluation loss improved. Best model overwritten.")
    else:
        patience_counter += 1
        print(f"Epoch {epoch+1}: No improvement. Patience counter: {patience_counter}/{patience}")
    
    # patience 초과 시 얼리 스토핑
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

    model.train()

Epoch 1: 100%|██████████| 7500/7500 [40:43<00:00,  3.07it/s, loss=1.6] 


Epoch 1 Training Loss: 0.8565


Evaluating: 100%|██████████| 282/282 [01:43<00:00,  2.73it/s]


Epoch 1 Evaluation Loss: 1.6165
Epoch 1: Evaluation loss improved. Best model overwritten.


Epoch 2: 100%|██████████| 7500/7500 [1:01:08<00:00,  2.04it/s, loss=1.89]


...

Epoch 4 Evaluation Loss: 1.4866
Epoch 4: Evaluation loss improved. Best model overwritten.


Epoch 5: 100%|██████████| 7500/7500 [57:32<00:00,  2.17it/s, loss=1.1]   


Epoch 5 Training Loss: 0.6718


Evaluating: 100%|██████████| 282/282 [01:23<00:00,  3.37it/s]


Epoch 5 Evaluation Loss: 1.4851
Epoch 5: Evaluation loss improved. Best model overwritten.


In [ ]:
# 최종 모델 저장
model.save_pretrained("./results")

# 마지막 LR 값 저장
last_lr = scheduler.get_last_lr()
print(f"Last Learning Rate: {last_lr}")  # 리스트 형태로 확인

# 파일에 저장할 때 형식 조정
with open("./results/last_lr.txt", "w") as f:
    f.write(f"Last Learning Rate: {last_lr[0]:.10f}\n")  # 소수점 10자리까지 저장


Last Learning Rate: [0.0]


## compute_metrics를 이용한 BLEU score 출력

In [99]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # 빠른 평가를 위해 greedy decoding 사용 (num_beams=1)
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=1,          # greedy decoding으로 속도 향상
            early_stopping=True
        )

        # 생성된 토큰 ID 배열 수집
        all_preds.append(outputs.cpu().numpy())
        
        # 라벨 처리: -100을 pad 토큰 ID로 대체 후 수집
        labels = batch["labels"]
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        all_labels.append(labels.cpu().numpy())

# 각 배치마다 시퀀스 길이가 다를 수 있으므로, 전체 데이터에서 최대 시퀀스 길이를 계산합니다.
max_seq_len = max(
    max(arr.shape[1] for arr in all_preds),
    max(arr.shape[1] for arr in all_labels)
)

# 각 배치의 예측과 라벨을 max_seq_len에 맞게 padding 처리합니다.
all_preds_padded = []
all_labels_padded = []
for pred, label in zip(all_preds, all_labels):
    pred_padded = np.pad(pred, ((0, 0), (0, max_seq_len - pred.shape[1])),
                         mode='constant', constant_values=tokenizer.pad_token_id)
    label_padded = np.pad(label, ((0, 0), (0, max_seq_len - label.shape[1])),
                          mode='constant', constant_values=tokenizer.pad_token_id)
    all_preds_padded.append(pred_padded)
    all_labels_padded.append(label_padded)

# 배치 축(axis=0) 기준으로 모든 배열을 하나로 합칩니다.
all_preds_padded = np.concatenate(all_preds_padded, axis=0)
all_labels_padded = np.concatenate(all_labels_padded, axis=0)

# compute_metrics 함수에 (preds, labels) 튜플을 전달하여 BLEU 점수를 계산합니다.
result = compute_metrics((all_preds_padded, all_labels_padded))
print("BLEU score:", result["bleu"])


Testing: 100%|██████████| 282/282 [07:42<00:00,  1.64s/it]


BLEU score: 28.06752238313831


In [81]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import sacrebleu  

# 테스트 데이터셋 DataLoader
test_batch_size = 32  
test_dataloader = DataLoader(
    tokenized_datasets["test"],
    batch_size=test_batch_size,
    shuffle=False,
    collate_fn=data_collator
)

# 모델을 evaluation 모드로 전환
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Beam Search + Diverse Beam Search + Sampling 적용
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=8,           # Beam 수를 8로 변경 (num_beams % num_beam_groups == 0 이어야 함)
            num_beam_groups=2,     # Diverse beam search를 위해 그룹 수 지정 (1보다 큰 정수)
            length_penalty=1.0,    # 문장 길이 균형 조정
            early_stopping=True,   # 조기 종료
            diversity_penalty=0.3, # 다양성 강화
            top_k=50,              # Top-k sampling 적용
            top_p=0.95,            # Top-p (nucleus) sampling 적용
            repetition_penalty=1.2 # 단어 반복 방지
        )

        # 생성된 토큰들을 디코딩
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend([pred.strip().lower() for pred in decoded_preds])

        # 정답 라벨 처리 (-100을 padding id로 변환)
        labels = batch["labels"]
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        references.extend([ref.strip().lower() for ref in decoded_refs])

# BLEU 점수 계산
bleu = sacrebleu.corpus_bleu(predictions, [references])
print("BLEU score:", bleu.score)

# 번역 결과 예시 3개 출력
print("\n번역 예시 3개:")
for i in range(3):
    print(f"예측  : {predictions[i]}")
    print(f"참조  : {references[i]}")
    print("-" * 40)


Testing: 100%|██████████| 32/32 [05:02<00:00,  9.44s/it]

BLEU score: 31.148002752322434

번역 예시 3개:
예측  : in the meantime, i'm bored to stay at home with my dog.
참조  : i'm bored to stay home with my dog.
----------------------------------------
예측  : lotte, which gave samsung's kang min-ho a solo home run, became 3-2, with chae tae-in's left-handed one-run home run and shin bon-ki's superior one-run home run in the top of the eighth inning without a runner.
참조  : lotte, which allowed a solo home run for samsung kang min-ho and became a 3-2 runner-up, had a rendezvous home run, including chae tae-in's left point home run and shin bon-ki's right point home run.
----------------------------------------
예측  : in order, i would like to send this letter by register.
참조  : i'd like to send this letter registered.
----------------------------------------


## 모델 테스트 (Test)
- 학습된 모델을 가지고 테스트 데이터로 테스트

In [95]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np

# 테스트 데이터셋 DataLoader 설정 (배치 크기 32)
test_batch_size = 32  
test_dataloader = DataLoader(
    tokenized_datasets["test"],
    batch_size=test_batch_size,
    shuffle=False,
    collate_fn=data_collator
)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # 빠른 평가를 위해 greedy decoding 사용 (num_beams=1)
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=2,          # greedy decoding으로 속도 향상
            early_stopping=True
        )

        # 생성된 토큰 ID 배열 수집
        all_preds.append(outputs.cpu().numpy())
        
        # 라벨 처리: -100을 pad 토큰 ID로 대체 후 수집
        labels = batch["labels"]
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        all_labels.append(labels.cpu().numpy())

# 각 배치마다 시퀀스 길이가 다를 수 있으므로, 전체 데이터에서 최대 시퀀스 길이를 계산합니다.
max_seq_len = max(
    max(arr.shape[1] for arr in all_preds),
    max(arr.shape[1] for arr in all_labels)
)

# 각 배치의 예측과 라벨을 max_seq_len에 맞게 padding 처리합니다.
all_preds_padded = []
all_labels_padded = []
for pred, label in zip(all_preds, all_labels):
    pred_padded = np.pad(pred, ((0, 0), (0, max_seq_len - pred.shape[1])),
                         mode='constant', constant_values=tokenizer.pad_token_id)
    label_padded = np.pad(label, ((0, 0), (0, max_seq_len - label.shape[1])),
                          mode='constant', constant_values=tokenizer.pad_token_id)
    all_preds_padded.append(pred_padded)
    all_labels_padded.append(label_padded)

# 배치 축(axis=0) 기준으로 모든 배열을 하나로 합칩니다.
all_preds_padded = np.concatenate(all_preds_padded, axis=0)
all_labels_padded = np.concatenate(all_labels_padded, axis=0)

# compute_metrics 함수에 (preds, labels) 튜플을 전달하여 BLEU 점수를 계산합니다.
result = compute_metrics((all_preds_padded, all_labels_padded))
print("BLEU score:", result["bleu"])


Testing: 100%|██████████| 32/32 [01:58<00:00,  3.71s/it]

BLEU score: 29.224891396155897


In [ ]:
# 상위 3개의 원문과 번역 결과 출력
for i in range(3):
    # 번역 결과 디코드
    decoded_pred = tokenizer.decode(all_preds_padded[i], skip_special_tokens=True)
    decoded_label = tokenizer.decode(all_labels_padded[i], skip_special_tokens=True)
    
    print(f"번역결과 {i+1}: {decoded_pred}")
    print(f"정답 {i+1}: {decoded_label}")
    print("="*50)

번역결과 1: In the meantime, I'm bored to stay at home with my dog.
정답 1: I'm bored to stay home with my dog.
번역결과 2: Lotte, which gave Samsung's Kang Min-ho a solo home run, became 3-2, with Chae Tae-in's left-field one-run home run and Shin Bon-ki's superior one-run home run in the top of the eighth inning without a runner.
정답 2: Lotte, which allowed a solo home run for Samsung Kang Min-ho and became a 3-2 runner-up, had a rendezvous home run, including Chae Tae-in's left point home run and Shin Bon-ki's right point home run.
번역결과 3: I'd like to send this letter to you.
정답 3: I'd like to send this letter registered.


## Inference
- Assignment2에 쓰였던 문장들을 이 학습된 모델에서 그 결과를 살펴 보아라

- 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
- 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
- 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요
- 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
- 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다
- 변기가 막혔습니다
- 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
- 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.
- 속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다
- 문대통령은 집단 이익에서 벗어나라고 말했다
- 이것 좀 먹어 볼 몇 일 간의 시간을 주세요
- 이 날 개미군단은 외인의 물량을 모두 받아 내었다
- 통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다
- 이번 구조 조정이 제품을 효과적으로 개발 하고 판매 하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다
- 요즘 이 프로그램 녹화하며 많은 걸 느낀다

In [105]:
# checkpoint 경로 지정
checkpoint_path = "./results"

In [106]:
# 체크포인트가 존재하면 모델 불러오기, 없으면 새로 초기화 (여기서는 체크포인트가 있다고 가정)
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}")
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
else:
    # 모델 초기화 코드 (예: model = MyModelClass.from_pretrained("pretrained-model-name"))
    raise ValueError("Checkpoint not found. Please provide a valid checkpoint path.")

Loading checkpoint from ./results


In [107]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [108]:
# 번역할 문장 리스트
sentences = [
    "모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.",
    "미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?",
    "은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요",
    "아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?",
    "부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다",
    "변기가 막혔습니다",
    "그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?",
    "비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.",
    "속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다",
    "문대통령은 집단 이익에서 벗어나라고 말했다",
    "이것 좀 먹어 볼 몇 일 간의 시간을 주세요",
    "이 날 개미군단은 외인의 물량을 모두 받아 내었다",
    "통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다",
    "이번 구조 조정이 제품을 효과적으로 개발 하고 판매 하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다",
    "요즘 이 프로그램 녹화하며 많은 걸 느낀다"
]

# 테스트 문장에 대한 번역
model.eval()
decoded_preds = []

with torch.no_grad():
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=2,
            early_stopping=True
        )

        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
        decoded_preds.append(decoded_output)

# 결과 출력
for i, sentence in enumerate(sentences):
    print(f"원문 {i+1}: {sentence}")
    print(f"번역결과 {i+1}: {decoded_preds[i]}")
    print("="*50)


원문 1: 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
번역결과 1: All liquids, gels, and airols should be placed in one foldable transparent box.
원문 2: 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
번역결과 2: I'm sorry, but the kids are talking so much that I want to go to Gwanghwamun, could you change the ticket?
원문 3: 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요
번역결과 3: I can't because the bank is too far away. If you need cash, grab the money.
원문 4: 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
번역결과 4: I think I lost it, so I need to fill out a loss report. Do you want to go to the office together?
원문 5: 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다
번역결과 5: The number of suspected patients in Busan has increased rapidly and there is a shortage of beds, so 20 confirmed patients will be transferred to Daegu.
원문 6: 변기가 막혔습니다
번역결과 6: The toilet was blocked.
원문 7: 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
번역결과 7: Please show me the pants. How much do I get them?
원문 8: 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.